In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2

from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms,datasets

from torchsummary import summary as summary_

In [52]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR_2D(in_ch, out_ch, k_size = 3, stride = 1, padding = 1, bias = True):
            layers =[]
            layers += [nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=k_size, stride=stride, padding=padding, bias=bias)]
            layers += [nn.BatchNorm2d(num_features=out_ch)]
            layers += [nn.ReLU()]

            conv_block = nn.Sequential(*layers)

            return conv_block

        self.encoder_layer_1_1 = CBR_2D(in_ch=3, out_ch=64) 
        self.encoder_layer_1_2 = CBR_2D(in_ch=64, out_ch=64)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.encoder_layer_2_1 = CBR_2D(in_ch=64, out_ch=128)
        self.encoder_layer_2_2 = CBR_2D(in_ch=128, out_ch=128)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.encoder_layer_3_1 = CBR_2D(in_ch=128, out_ch=256)
        self.encoder_layer_3_2 = CBR_2D(in_ch=256, out_ch=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.decoder_layer_4_2 = CBR_2D(in_ch=256, out_ch=512)
        self.decoder_layer_4_1 = CBR_2D(in_ch=512, out_ch=512)

        self.unpool3 = nn.ConvTranspose2d(in_channels=512, out_channels=256, \
                                        kernel_size=2, stride=2, padding=0, bias=True)
        
        self.decoder_layer_3_2 = CBR_2D(in_ch=512, out_ch=256)
        self.decoder_layer_3_1 = CBR_2D(in_ch=256, out_ch=256)


        self.unpool2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, \
                                        kernel_size=2, stride=2, padding=0, bias=True)

        self.decoder_layer_2_2 = CBR_2D(in_ch=256, out_ch=128)
        self.decoder_layer_2_1 = CBR_2D(in_ch=128, out_ch=128)

        self.unpool1 = nn.ConvTranspose2d(in_channels=128, out_channels=64, \
                                        kernel_size=2, stride=2, padding=0, bias=True)

        self.decoder_layer_1_2 = CBR_2D(in_ch=128, out_ch=64)
        self.decoder_layer_1_1 = CBR_2D(in_ch=64, out_ch=64)
        self.conv1x1 = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=1)

    def forward(self, x):
        enc1_1 = self.encoder_layer_1_1(x)
        enc1_2 = self.encoder_layer_1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.encoder_layer_2_1(pool1)
        enc2_2 = self.encoder_layer_2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.encoder_layer_3_1(pool2)
        enc3_2 = self.encoder_layer_3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        bottom = self.decoder_layer_4_2(pool3)
        bottom = self.decoder_layer_4_1(bottom)
        unpool3 = torch.cat((self.unpool3(bottom), enc3_2), dim=1)

        dec3_2 = self.decoder_layer_3_2(unpool3)
        dec3_1 = self.decoder_layer_3_1(dec3_2)
        unpool2 = torch.cat((self.unpool2(dec3_1), enc2_2), dim=1)

        dec2_2 = self.decoder_layer_2_2(unpool2)
        dec2_1 = self.decoder_layer_2_1(dec2_2)
        unpool1 = torch.cat((self.unpool1(dec2_1), enc1_2), dim=1)

        dec1_2 = self.decoder_layer_1_2(unpool1)
        dec1_1 = self.decoder_layer_1_1(dec1_2)

        output = self.conv1x1(dec1_1)

        return output

model = UNet()
model.cuda()

summary_(model, (3, 256, 256), batch_size= 1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 256, 256]           1,792
       BatchNorm2d-2          [1, 64, 256, 256]             128
              ReLU-3          [1, 64, 256, 256]               0
            Conv2d-4          [1, 64, 256, 256]          36,928
       BatchNorm2d-5          [1, 64, 256, 256]             128
              ReLU-6          [1, 64, 256, 256]               0
         MaxPool2d-7          [1, 64, 128, 128]               0
            Conv2d-8         [1, 128, 128, 128]          73,856
       BatchNorm2d-9         [1, 128, 128, 128]             256
             ReLU-10         [1, 128, 128, 128]               0
           Conv2d-11         [1, 128, 128, 128]         147,584
      BatchNorm2d-12         [1, 128, 128, 128]             256
             ReLU-13         [1, 128, 128, 128]               0
        MaxPool2d-14           [1, 128,

In [10]:
def CBR_2D(in_ch, out_ch, k_size = 3, stride = 1, padding = 1, bias = True):
    layers =[]
    layers += [nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=k_size, stride=stride, padding=padding, bias=bias)]
    layers += [nn.BatchNorm2d(num_features=out_ch)]
    layers += [nn.ReLU()]

    conv_block = nn.Sequential(*layers)

    return conv_block

input = torch.randn(20, 3, 256, 256)
output = CBR_2D(3, 64)(input)
output = CBR_2D(64, 64)(output)
output = nn.MaxPool2d(kernel_size=2)(output)

output = CBR_2D(64, 128)(output)
output = CBR_2D(128, 128)(output)
output = nn.MaxPool2d(kernel_size=2)(output)

output = CBR_2D(128, 256)(output)
output = CBR_2D(256, 256)(output)
output = nn.MaxPool2d(kernel_size=2)(output)

output = CBR_2D(256, 512)(output)
output = CBR_2D(512, 512)(output)

output = nn.ConvTranspose2d(in_channels=512, out_channels=256, \
                                        kernel_size=2, stride=2, padding=0, bias=True)(output)

output = 

output.size()

torch.Size([20, 256, 64, 64])

In [41]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform

        data = os.listdir(self.data_dir)

        img_list, label_list = [], []
        for f in sorted(data):
            if 'sample' in f:
                img_list.append(f) 
            elif 'label' in f:
                label_list.append(f)

        self.img_list = img_list
        self.label_list = label_list

    def __len__(self):
        return len(self.label_list)

    def __getitem__(self, index):

        input = cv2.imread(os.path.join(self.data_dir, self.img_list[index]))
        label = cv2.imread(os.path.join(self.data_dir, self.label_list[index]), cv2.IMREAD_GRAYSCALE)

        input, label = input/255.0, label/255.0
        input, label = input.astype(np.float32), label.astype(np.float32)

        if label.ndim == 2:
            label = label[:, :, np.newaxis]
        if input.ndim == 2:
            input = input[:, :, np.newaxis]

        data = {'input': input, 'label': label}

        if self.transform:
            data = self.transform(data)

        return data

In [33]:
class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        label = label.transpose((2, 0, 1)).astype(np.float32)
        input = input.transpose((2, 0, 1)).astype(np.float32)

        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        return data

class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        label, input = data['label'], data['input']

        input = (input - self.mean) / self.std

        data = {'label': label, 'input': input}

        return data

class RandomFlip(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        if np.random.rand() > 0.5:
            label = np.fliplr(label)
            input = np.fliplr(input)

        if np.random.rand() > 0.5:
            label = np.flipud(label)
            input = np.flipud(input)

        data = {'label': label, 'input': input}

        return data

In [53]:
lr = 1e-3
batch_size = 4
num_epoch = 100

data_dir = '/mnt/hdd1/c-MET_datasets/Lung_c-MET IHC_scored/DL-based_tumor_seg_dataset'
ckpt_dir = '/mnt/hdd1/model/Lung_c-MET IHC_scored/UNet/checkpoint'
log_dir = '/mnt/hdd1/model/Lung_c-MET IHC_scored/UNet/log'

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else cpu)

In [62]:
transform_train = transforms.Compose([Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])
dataset_train = Dataset(data_dir=os.path.join(data_dir,'train'),transform = transform_train)

loader_train = DataLoader(dataset_train, batch_size = batch_size, shuffle = True)

transform_val = transforms.Compose([ToTensor()])
dataset_val = Dataset(data_dir=os.path.join(data_dir, 'valid'), transform = transform_val)
loader_val = DataLoader(dataset_val, batch_size = batch_size, shuffle = True)

net = UNet().to(device)

fn_loss = nn.BCEWithLogitsLoss().to(device)

optim = torch.optim.Adam(net.parameters(), lr = lr)

num_train = len(dataset_train)
num_val = len(dataset_val)

num_train_for_epoch = np.ceil(num_train/batch_size)
num_val_for_epoch = np.ceil(num_val/batch_size)

fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0, 2, 3, 1)
fn_denorm = lambda x, mean, std : (x*std) + mean
fn_classifier = lambda x : 1.0 * (x > 0.5)

writer_train = SummaryWriter(log_dir = os.path.join(log_dir, 'train'))
writer_val = SummaryWriter(log_dir = os.path.join(log_dir, 'valid'))


In [ ]:
filter(str, f)

In [63]:
# 네트워크 저장하기
# train을 마친 네트워크 저장 
# net : 네트워크 파라미터, optim  두개를 dict 형태로 저장
def save(ckpt_dir,net,optim,epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net':net.state_dict(),'optim':optim.state_dict()},'%s/model_epoch%d.pth'%(ckpt_dir,epoch))

# 네트워크 불러오기
def load(ckpt_dir,net,optim):
    if not os.path.exists(ckpt_dir): # 저장된 네트워크가 없다면 인풋을 그대로 반환
        epoch = 0
        return net, optim, epoch
    
    ckpt_lst = os.listdir(ckpt_dir) # ckpt_dir 아래 있는 모든 파일 리스트를 받아온다
    ckpt_lst.sort(key = lambda f : int(''.join(filter(str.isdigit,f))))

    dict_model = torch.load('%s/%s' % (ckpt_dir,ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])
    epoch = int(ckpt_lst[-1].split('epoch')[1].split('.pth')[0])

    return net,optim,epoch


# 네트워크 학습시키기
start_epoch = 0
net, optim, start_epoch = load(ckpt_dir = ckpt_dir, net = net, optim = optim) # 저장된 네트워크 불러오기

for epoch in range(start_epoch+1,num_epoch +1):
    net.train()
    loss_arr = []

    for batch, data in enumerate(loader_train,1): # 1은 뭐니 > index start point
        # forward
        label = data['label'].to(device)   # 데이터 device로 올리기     
        inputs = data['input'].to(device)
        output = net(inputs) 

        # backward
        optim.zero_grad()  # gradient 초기화
        loss = fn_loss(output, label)  # output과 label 사이의 loss 계산
        loss.backward() # gradient backpropagation
        optim.step() # backpropa 된 gradient를 이용해서 각 layer의 parameters update

        # save loss
        loss_arr += [loss.item()]

        # tensorbord에 결과값들 저장하기
        label = fn_tonumpy(label)
        inputs = fn_tonumpy(fn_denorm(inputs,0.5,0.5))
        output = fn_tonumpy(fn_classifier(output))

        writer_train.add_image('label', label, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('input', inputs, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
        writer_train.add_image('output', output, num_train_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

    writer_train.add_scalar('loss', np.mean(loss_arr), epoch)

    
    # validation
    with torch.no_grad(): # validation 이기 때문에 backpropa 진행 x, 학습된 네트워크가 정답과 얼마나 가까운지 loss만 계산
        net.eval() # 네트워크를 evaluation 용으로 선언
        loss_arr = []

        for batch, data in enumerate(loader_val,1):
            # forward
            label = data['label'].to(device)
            inputs = data['input'].to(device)
            output = net(inputs)

            # loss 
            loss = fn_loss(output,label)
            loss_arr += [loss.item()]
            print('valid : epoch %04d / %04d | Batch %04d \ %04d | Loss %04d'%(epoch,num_epoch,batch,num_val_for_epoch,np.mean(loss_arr)))

            # Tensorboard 저장하기
            label = fn_tonumpy(label)
            # inputs = fn_tonumpy(fn_denorm(inputs, mean=0.5, std=0.5))
            output = fn_tonumpy(fn_classifier(output))

            writer_val.add_image('label', label, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('input', inputs, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')
            writer_val.add_image('output', output, num_val_for_epoch * (epoch - 1) + batch, dataformats='NHWC')

        writer_val.add_scalar('loss', np.mean(loss_arr), epoch)

        # epoch이 끝날때 마다 네트워크 저장
        save(ckpt_dir=ckpt_dir, net = net, optim = optim, epoch = epoch)

writer_train.close()
writer_val.close()

valid : epoch 0001 / 0100 | Batch 0001 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0002 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0003 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0004 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0005 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0006 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0007 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0008 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0009 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0010 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0011 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0012 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0013 \ 0014 | Loss 0000
valid : epoch 0001 / 0100 | Batch 0014 \ 0014 | Loss 0000
valid : epoch 0002 / 0100 | Batch 0001 \ 0014 | Loss 0000
valid : epoch 0002 / 0100 | Batch 0002 \ 0014 | Loss 0000
valid : epoch 0002 / 0100 | Batch 0003 \ 0014 | Loss 0000
valid : epoch 

In [64]:
%load_ext tensorboard

In [65]:
tensorboard --logdir='/mnt/hdd1/model/Lung_c-MET IHC_scored/UNet/log'

Reusing TensorBoard on port 6007 (pid 48776), started 3:05:50 ago. (Use '!kill 48776' to kill it.)